# Imports

In [4]:
# Basics
import numpy as np
import pandas as pd
import nltk
import unicodedata
import re

# Visuals
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

plt.rc('figure', figsize=(13, 7))
plt.style.use('seaborn-darkgrid')

# Exercise 1
**Spam Data**

   - Load the spam data set.
   - Create and explore bigrams for the spam data. Visualize them with a word cloud. How do they compare with the ham bigrams?
   - Is there any overlap in the bigrams for the spam data and the ham data?
   - Create and explore with trigrams (i.e. a n-gram with an n of 3) for both the spam and ham data.



# Exercise 2


**Explore the blog articles using the techniques discussed in the exploration lesson.**


# Exercise 3
**Explore the news articles using the techniques discussed in the exploration lesson. Use the category variable when exploring.**